## First Question : scarp top 250 movie imbd with information

### import libraries

In [806]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re

##### get main url

In [807]:
urlmain = 'https://www.imdb.com/chart/top/'


#### Send get request to url 

In [808]:
headers = {'Accept-Language': 'en-US',"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [809]:
response = requests.get(url=urlmain , headers=headers)

In [810]:
print(response.status_code)

200


##### parse content

In [811]:
soup = BeautifulSoup(response.content, 'html.parser')

get link for each movies

In [812]:
links_250 = soup.find_all('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b85248f1-7 lhgKeb cli-title')

In [813]:
scrap_info_movies = []

In [814]:
session = requests.Session()
for link in links_250: 
    a_element = link.find("a")
    get_att = a_element.get('href')
    movie_urls = f"https://www.imdb.com{get_att}"
    response_each_movie = requests.get(url=movie_urls, headers=headers)
    movies_soup = BeautifulSoup(response_each_movie.content, 'html.parser')
    titles = movies_soup.find('span', class_='sc-afe43def-1 fDTGTb').text
    film_id = re.search(r'/tt(\d+)/', get_att)
    if film_id:
        film_id = film_id.group(1)
    else:
        film_id = None
    gross_us_canada_element = movies_soup.find('li', {'data-testid': 'title-boxoffice-grossdomestic'})
    gross_us_canada = None
    if gross_us_canada_element:
        span_element = gross_us_canada_element.find('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        if span_element:
            gross_us_canada = span_element.text
    ul_element = movies_soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'})
    years = None
    parental_guide = None
    runtime = None
    for li_element in ul_element.find_all('li', {'class': 'ipc-inline-list__item'}):
        ainner_element = li_element.find('a', {'class': 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'})
        if ainner_element:
            href = ainner_element.get('href')
            text = ainner_element.text
            if 'releaseinfo' in href:
                years = text
            elif 'parentalguide' in href:
                parental_guide = text
        else:
            runtime = li_element.text
    genre_list = movies_soup.find_all('a', {'class': 'ipc-chip ipc-chip--on-baseAlt'})
    genre = [genre.find('span', class_='ipc-chip__text').text for genre in genre_list]
    #crew
    ul_crew = movies_soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt')
    directors = None
    stars = []
    writers = []
    person_id = []
    for li_element in ul_crew.find_all('li', {'class': 'ipc-inline-list__item'}):
        ainner_crew = li_element.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        if ainner_crew:
            href = ainner_crew.get('href')
            text = ainner_crew.text
            if 'tt_ov_wr' in href:
                writers.append(text)
            elif 'tt_ov_st' in href:
                stars.append(text)
            elif 'tt_ov_dr' in href:
                directors = li_element.text
            person_id_match = re.search(r'/name/nm(\d+)/', href)
            if person_id_match:
                person_ids = person_id_match.group(1)
                person_id.append(person_ids)
    scrap_info_movies.append({'title': titles,'runtime':runtime, 'parental_guide' : parental_guide, 'genre': genre, 'stars': stars, 'directors': directors, 'writers': writers , 'gross_us_canada':gross_us_canada , 'film_id':film_id,'person_id':person_id})
    time.sleep(1)

session.close()


In [818]:
top_250 = pd.DataFrame(scrap_info_movies)

In [819]:
top_250.head(50)

,title,runtime,parental_guide,genre,stars,directors,writers,gross_us_canada,film_id,person_id
0,The Shawshank Redemption,2h 22m,R,[Drama],"[Tim Robbins, Morgan Freeman, Bob Gunton]",Frank Darabont,"[Stephen King, Frank Darabont]","$28,767,189",0111161,"[0001104, 0000175, 0001104, 0000209, 0000151, ..."
1,The Godfather,2h 55m,R,"[Crime, Drama]","[Marlon Brando, Al Pacino, James Caan]",Francis Ford Coppola,"[Mario Puzo, Francis Ford Coppola]","$136,381,073",0068646,"[0000338, 0701374, 0000338, 0000008, 0000199, ..."
2,The Dark Knight,2h 32m,PG-13,"[Action, Crime, Drama]","[Christian Bale, Heath Ledger, Aaron Eckhart]",Christopher Nolan,"[Jonathan Nolan, Christopher Nolan, David S. G...","$534,987,076",0468569,"[0634240, 0634300, 0634240, 0275286, 0000288, ..."
3,The Godfather Part II,3h 22m,R,"[Crime, Drama]","[Al Pacino, Robert De Niro, Robert Duvall]",Francis Ford Coppola,"[Francis Ford Coppola, Mario Puzo]","$47,834,595",0071562,"[0000338, 0000338, 0701374, 0000199, 0000134, ..."
4,12 Angry Men,1h 36m,Approved,"[Crime, Drama]","[Henry Fonda, Lee J. Cobb, Martin Balsam]",Sidney Lumet,[Reginald Rose],None,0050083,"[0001486, 0741627, 0000020, 0002011, 0000842]"
5,Schindler's List,3h 15m,R,"[Biography, Drama, History]","[Liam Neeson, Ralph Fiennes, Ben Kingsley]",Steven Spielberg,"[Thomas Keneally, Steven Zaillian]","$96,898,818",0108052,"[0000229, 0447745, 0001873, 0000553, 0000146, ..."
6,The Lord of the Rings: The Return of the King,3h 21m,PG-13,"[Action, Adventure, Drama]","[Elijah Wood, Viggo Mortensen, Ian McKellen]",Peter Jackson,"[J.R.R. Tolkien, Fran Walsh, Philippa Boyens]","$379,427,292",0167260,"[0001392, 0866058, 0909638, 0101991, 0000704, ..."
7,Pulp Fiction,2h 34m,R,"[Crime, Drama]","[John Travolta, Uma Thurman, Samuel L. Jackson]",Quentin Tarantino,"[Quentin Tarantino, Roger Avary]","$107,928,762",0110912,"[0000233, 0000233, 0000812, 0000237, 0000235, ..."
8,The Lord of the Rings: The Fellowship of the Ring,2h 58m,PG-13,"[Action, Adventure, Drama]","[Elijah Wood, Ian McKellen, Orlando Bloom]",Peter Jackson,"[J.R.R. Tolkien, Fran Walsh, Philippa Boyens]","$316,115,420",0120737,"[0001392, 0866058, 0909638, 0101991, 0000704, ..."
9,"The Good, the Bad and the Ugly",2h 58m,Approved,"[Adventure, Western]","[Clint Eastwood, Eli Wallach, Lee Van Cleef]",Sergio Leone,"[Luciano Vincenzoni, Sergio Leone, Agenore Inc...","$25,100,000",0060196,"[0001466, 0898812, 0001466, 0408488, 0000142, ..."


In [820]:
top_250.to_csv('top_250.csv', index=False)

In [ ]:
"""     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    directore = [directore.get_text() for directore in directore_list]
    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    writer = 
    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    star = 
    gross_us_canada =   """

"     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    directore = [directore.get_text() for directore in directore_list]\n    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    writer = \n    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    star = \n    gross_us_canada =   "